In [1]:
%load_ext autoreload
%autoreload 2

# Process Data

In [24]:
import joblib, random, torch

random.seed(11)
pickle_path = "data/rectified_data/dataset_fog_release/val_dataset_fog_release.p"

all_data = joblib.load(pickle_path)

print(len(all_data.keys()))
print(all_data[200].keys())

gt = all_data[200]['gt']
print(all_data[200]['series_name'])
print(gt.shape)
print(gt[-3:])

285
dict_keys(['series_name', 'start_t_idx', 'end_t_idx', 'model_input', 'gt'])
rectified_16_dataset_fog_release
torch.Size([15552])
tensor([2, 2, 2], dtype=torch.int8)


In [4]:

import torch
bce = torch.nn.BCELoss(reduction='none')

pred = torch.rand(2,6)
mask = torch.tensor([[0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 1]]).float()

loss = bce(pred, mask)
loss.shape, loss


(torch.Size([2, 6]),
 tensor([[1.7707, 2.4249, 0.1701, 0.8715, 2.5939, 1.6950],
         [1.1421, 0.6567, 0.1020, 0.8278, 0.0357, 1.0962]]))

# Overfit training example

In [38]:
class DictToObj:
    def __init__(self, dictionary):
        self.__dict__.update(dictionary)
    
    def __getattr__(self, name):
        return self.__dict__.get(name)
    
opt = {
    'seed': 11,
    'optimizer': 'adamw',
    'learning_rate': 0.00026,
    'adam_betas': (0.9, 0.98),
    'adam_eps': 1.0e-09,
    'weight_decay': 0,
    'lr_scheduler_factor': 0.1,
    'lr_scheduler_patience': 20,
    'lr_scheduler_warmup_steps': 64,
    'train_num_steps': 20000,
    'penalty_cost': 2.0,
    'block_size': 15552,
    'block_stride': 972,
    'patch_size': 18,
    'fog_model_input_dim': 162,
    'fog_model_dim': 320,
    'fog_model_num_heads': 8,
    'fog_model_num_encoder_layers': 5,
    'fog_model_num_lstm_layers': 2,
    'fog_model_first_dropout': 0.1,
    'fog_model_encoder_dropout': 0.1,
    'fog_model_mha_dropout': 0.0,
}
opt = DictToObj(opt)

import torch
import numpy as np
import random
from models.transformer_bilstm_v1 import TransformerBiLSTM

random.seed(opt.seed)
np.random.seed(opt.seed)
torch.manual_seed(opt.seed)

model = TransformerBiLSTM(opt)


#### Train

In [18]:

from data.fog_dataset_v1 import FoGDataset

train_dpath = "data/rectified_data/dataset_fog_release/train1_dataset_fog_release_blks15552_ps18.p"
model_path = "runs/train/transfomer_bilstm/2024_06_12_21:12:41.10/weights/model_regular_19991.pt"

model.load_state_dict(torch.load(model_path)['model'])


<All keys matched successfully>

In [19]:
import joblib
train_ds = joblib.load(train_dpath)
len(train_ds.keys()), train_ds[1].keys(), train_ds[1]['model_input'].shape, train_ds[1]['gt'].shape

(1055,
 dict_keys(['series_name', 'start_t_idx', 'end_t_idx', 'model_input', 'gt']),
 torch.Size([864, 162]),
 torch.Size([864, 3]))

In [20]:
model_input = []
gt = []
for i in range(32):
    model_input.append(train_ds[i]['model_input'][None, :, :])
    gt.append(train_ds[i]['gt'][None, :, :])

model_input= torch.cat(model_input, dim=0)
gt = torch.cat(gt, dim=0)
model_input.shape, gt.shape

(torch.Size([32, 864, 162]), torch.Size([32, 864, 3]))

In [31]:
bce_loss = torch.nn.BCELoss(reduction='none')

penalty_cost = 2.0

def loss_func(pred, gt):
        """Compute the Binary Cross-Entropy loss for each class and sum over the class dimension

        Args:
            pred: (B, BLKS//P, 2)
            gt: (B, BLKS//P, 3)
        """
        loss = bce_loss(pred, gt[:,:,:2]) # (B, BLKS//P, 2)
        mask = (gt[:,:,2] != 1).float() # (B, BLKS//P)
        mask = mask.unsqueeze(-1).expand(-1, -1, 2) # (B, BLKS//P, 2)
        
        # Additional cost for misclassifying the minority class
        minority_mask = (gt[:,:,1] == 1).float() # (B, BLKS//P)
        minority_mask = minority_mask.unsqueeze(-1).expand(-1, -1, 2) # (B, BLKS//P, 2)
        loss = loss * (mask + penalty_cost * minority_mask)
   
        return loss.sum() / mask.sum()

In [34]:
def evaluation_metrics(output, gt):
        """Generate precision, recall, and f1 score.

        Args:
            output: (B, BLKS//P, 2)   # prob class
            gt:   (B, BLKS//P, 3)   # the third pos indicates the class
        """
        # Convert the model output probabilities to class predictions
        pred = torch.argmax(output, dim=-1)  # (B, BLKS//P)

        # Extract the first two classes from the ground truth
        real = torch.argmax(gt[:, :, :2], dim=-1)  # (B, BLKS//P)

        # Create a mask to ignore the positions where the ground truth class is 2
        mask = (gt[:, :, 2] != 1)  # (B, BLKS//P)

        # Apply the mask to the predictions and ground truth
        pred = pred * mask.long() # (B, BLKS//P)
        real = real * mask.long() # (B, BLKS//P)

        # Calculate true positives, false positives, and false negatives
        tp = ((pred == 1) & (real == 1)).float().sum()
        fp = ((pred == 1) & (real == 0)).float().sum()
        fn = ((pred == 0) & (real == 1)).float().sum()

        # Calculate precision, recall, and F1 score
        precision = tp / (tp + fp + 1e-8)
        recall = tp / (tp + fn + 1e-8)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
        
        # Calculate mean average precision (mAP)
        # Create a list to store the precision values at different thresholds
        precisions = []
        recalls = []
        
        # Iterate over thresholds from 0 to 1 with a step size
        thresholds = torch.linspace(0, 1, steps=101)
        for t in thresholds:
            # Binarize predictions based on the threshold
            pred_binary = (output[:, :, 1] >= t).float()
            
            # Apply the mask to the binary predictions and ground truth
            pred_binary = pred_binary * mask # (B, BLKS//P)
            real_binary = real * mask        
            
            # Calculate true positives, false positives, and false negatives
            tp_t = ((pred_binary == 1) & (real_binary == 1)).float().sum()
            fp_t = ((pred_binary == 1) & (real_binary == 0)).float().sum()
            fn_t = ((pred_binary == 0) & (real_binary == 1)).float().sum()

            # Calculate precision and recall for the current threshold
            precision_t = tp_t / (tp_t + fp_t + 1e-8)
            recall_t = tp_t / (tp_t + fn_t + 1e-8)
            
            precisions.append(precision_t)
            recalls.append(recall_t)
        
        # Convert lists to tensors
        precisions = torch.tensor(precisions)
        recalls = torch.tensor(recalls)
        
        # Sort by recall
        recall_sorted, indices = torch.sort(recalls)
        precision_sorted = precisions[indices]
        
        # Calculate average precision
        ap = torch.trapz(precision_sorted, recall_sorted)
        
        return precision, recall, f1, ap

In [21]:
pred = model(model_input)

In [32]:
loss = loss_func(pred, gt)

In [33]:
loss

tensor(0.0006, grad_fn=<DivBackward0>)

In [35]:
precision, recall, f1, ap = evaluation_metrics(pred, gt)
precision, recall, f1, ap 

(tensor(0.9565), tensor(1.), tensor(0.9778), tensor(0.2577))

#### Validation

In [39]:
from data.fog_dataset_v1 import FoGDataset

val_dpath = "data/rectified_data/dataset_fog_release/val1_dataset_fog_release_blks15552_ps18.p"
model_path = "runs/train/transfomer_bilstm/2024_06_12_21:12:41.10/weights/model_regular_19991.pt"


model.load_state_dict(torch.load(model_path)['model'])
import joblib
val_ds = joblib.load(val_dpath)

model_input = []
gt = []
for i in range(32):
    model_input.append(val_ds[i]['model_input'][None, :, :])
    gt.append(val_ds[i]['gt'][None, :, :])

model_input= torch.cat(model_input, dim=0)
gt = torch.cat(gt, dim=0)

model_input.shape, gt.shape

(torch.Size([32, 864, 162]), torch.Size([32, 864, 3]))

In [40]:
pred = model(model_input)
loss = loss_func(pred, gt)
precision, recall, f1, ap = evaluation_metrics(pred, gt)
precision, recall, f1, ap, loss

(tensor(0.7070),
 tensor(0.5953),
 tensor(0.6464),
 tensor(0.4415),
 tensor(4.1332, grad_fn=<DivBackward0>))

#### Data Augmentation

In [9]:
import math

def norm_axis(a,b,c):
    newa=a/(math.sqrt(float(a*a+b*b+c*c)))
    newb=b/(math.sqrt(float(a*a+b*b+c*c)))
    newc=c/(math.sqrt(float(a*a+b*b+c*c)))
    return ([newa,newb,newc])

def rotation_matrix(axis, theta):
    axis = np.asarray(axis)
    axis = axis/math.sqrt(np.dot(axis, axis))
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)], 
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)], 
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

def rotateC(image,theta,a,b,c): ## theta: angle, a, b, c, eular vector
    axis=norm_axis(a,b,c)
    imagenew=np.dot(image, rotation_matrix(axis,theta))
    return imagenew


In [10]:
from data.fog_dataset_v1 import FoGDataset

train_dpath = "data/rectified_data/dataset_fog_release/train1_dataset_fog_release_blks15552_ps18.p"
import joblib
train_ds = joblib.load(train_dpath)

model_input = []
gt = []
for i in range(32):
    model_input.append(train_ds[i]['model_input'][None, :, :])
    gt.append(train_ds[i]['gt'][None, :, :])

model_input= torch.cat(model_input, dim=0)
gt = torch.cat(gt, dim=0)
model_input.shape, gt.shape

(torch.Size([32, 864, 162]), torch.Size([32, 864, 3]))

In [11]:
model_input = model_input.reshape(32,864, 18, 9)
model_input.shape

torch.Size([32, 864, 18, 9])

In [12]:
model_input = model_input.reshape(32, 864, 18, 3, 3)
model_input.shape

torch.Size([32, 864, 18, 3, 3])

In [15]:
import random
import numpy as np

theta = random.random()*math.pi*2
theta = random.random()*360
a=random.random()
b=random.random()
c=random.random()
axis=norm_axis(a,b,c)
tmp = rotation_matrix(axis,theta)
tmp.shape, tmp, theta

((3, 3),
 array([[ 0.38501953, -0.68464825,  0.61888345],
        [ 0.91940947,  0.22620226, -0.32174332],
        [ 0.08028816,  0.69288477,  0.71656438]]),
 215.03446095139176)

In [16]:
random.seed(11)
np.random.seed(11)
torch.manual_seed(11)

theta = random.random()*math.pi*2
theta = random.random()*360
a=random.random()
b=random.random()
c=random.random()

print(model_input[0,:2,:,:])
print()
tmp=rotateC(model_input.cpu().detach().numpy(),theta,a,b,c)
print(tmp[0,:2,:,:])
tmp.shape

tensor([[[[-0.6030, -0.3699, -0.0446],
          [-0.6301, -0.0343, -1.0127],
          [-0.6837,  0.2872,  0.8251]],

         [[-0.6274, -0.3699, -0.0446],
          [-0.6479,  0.0098, -0.9907],
          [-0.6837,  0.0158,  0.6969]],

         [[-0.6274, -0.3699, -0.0446],
          [-0.6479,  0.0539, -0.9907],
          [-0.7707, -0.0684,  0.6969]],

         [[-0.5542, -0.3425, -0.1125],
          [-0.6479,  0.0098, -0.9907],
          [-0.7707, -0.0684,  0.6969]],

         [[-0.5542, -0.3425, -0.1125],
          [-0.6301, -0.0343, -0.9443],
          [-0.7137,  0.0158,  0.5687]],

         [[-0.5542, -0.3972, -0.0786],
          [-0.6301,  0.0098, -0.9907],
          [-0.7137,  0.0158,  0.5012]],

         [[-0.5542, -0.3972, -0.0786],
          [-0.6123,  0.0098, -1.0347],
          [-0.7437, -0.1619,  0.6294]],

         [[-0.5786, -0.3972, -0.0786],
          [-0.6301, -0.0829, -0.9907],
          [-0.7437, -0.2555,  0.7576]],

         [[-0.5786, -0.3972, -0.0786],
         

(32, 864, 18, 3, 3)

#### Check ratio of two classes

In [16]:
import torch

a= torch.randint(0,9, size=(4,3))
a, a.permute(1,0)

(tensor([[6, 8, 1],
         [6, 3, 1],
         [1, 0, 1],
         [0, 4, 3]]),
 tensor([[6, 6, 1, 0],
         [8, 3, 0, 4],
         [1, 1, 1, 3]]))

In [17]:
import joblib, torch

all_dpath = "data/rectified_data/kaggle_pd_data/train_kaggle_pd_data_blks15552_ps18_randomaug.p"
all_data = joblib.load(all_dpath)
len(all_data.keys())

20552

In [21]:
for key, value in all_data.items():
    if value['gt'].shape[1] != 3:
        print(key)
all_data[0]['gt'].shape

torch.Size([864, 3])

In [9]:
# Function to calculate the ratio
def calculate_class_ratio(train_data):
    ones = 0
    zeros = 0
    for example_id, example_data in train_data.items():
        gt = example_data['gt']
        num_zeros = torch.sum(gt == 0).item()
        num_ones = torch.sum(gt == 1).item()
        num_twos = torch.sum(gt == 2).item()
        ones += num_ones
        zeros += num_zeros
    print("one: ", ones)
    print("zero: ", zeros)
    print("one to zero: ", zeros / ones)

# Calculate the ratio for the training data
calculate_class_ratio(all_data)


one:  3376251
zero:  11171255
one to zero:  3.3087750288707802
